In [18]:
from facenet_pytorch import InceptionResnetV1, training, fixed_image_standardization
from torch.utils.tensorboard import SummaryWriter
import torch
import torchvision
import pandas as pd
import numpy as np
import os
from PIL import Image


In [19]:
batch_size = 8
epochs = 16
workers = 0 if os.name == 'nt' else 8
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


# Load Data

In [20]:
data = pd.read_pickle('./race_prediction_image/data.pkl')
data['race'] = np.where(
    data['race'] == 4, 0.0, data['race']
)
data['race'] = np.where(
    data['race'] == 5, np.nan, data['race']
)
data = data.dropna()
print(data['race'].value_counts())
data.head()

0.0    2726
1.0     318
2.0     200
3.0     112
Name: race, dtype: int64


,id,race,img_path,absolute_img_path,cropped_path
0,12488.0,0.0,profile pics/60147.jpeg,./data/profile pics/60147.jpeg,./data/cropped/60147.jpeg
1,719703.0,0.0,profile pics/60148.jpeg,./data/profile pics/60148.jpeg,./data/cropped/60148.jpeg
2,722153.0,3.0,profile pics/60149.jpeg,./data/profile pics/60149.jpeg,./data/cropped/60149.jpeg
5,811618.0,3.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg
6,822540.0,0.0,profile pics/60153.jpeg,./data/profile pics/60153.jpeg,./data/cropped/60153.jpeg


In [21]:
def load_images(input):
    if os.path.exists(input):
        tmp = Image.open(input)
        test = tmp.getbands()
        keep = tmp.copy()
        return keep
    else: return pd.NA

In [22]:
data['face'] = data['cropped_path'].apply(load_images)
data = data.dropna()
print(data.shape)
data.head()

(2385, 6)


,id,race,img_path,absolute_img_path,cropped_path,face
5,811618.0,3.0,profile pics/60152.jpeg,./data/profile pics/60152.jpeg,./data/cropped/60152.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
7,865071.0,0.0,profile pics/60154.jpeg,./data/profile pics/60154.jpeg,./data/cropped/60154.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
8,988211.0,0.0,profile pics/60155.jpeg,./data/profile pics/60155.jpeg,./data/cropped/60155.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
9,1025311.0,0.0,profile pics/60156.jpeg,./data/profile pics/60156.jpeg,./data/cropped/60156.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...
10,1143891.0,3.0,profile pics/60157.jpeg,./data/profile pics/60157.jpeg,./data/cropped/60157.jpeg,<PIL.Image.Image image mode=RGB size=160x160 a...


# Build dataset and dataloader

In [23]:
tmp = data[['face','race']].reset_index(drop=True)
white = tmp[tmp['race'] == 0.0].sample(frac=0.9,random_state=2021)
tmp = tmp.drop(white.index)
x = tmp['face'].tolist()
y = tmp['race'].tolist()
tmp['race'].value_counts()

1.0    224
0.0    195
2.0    136
3.0     74
Name: race, dtype: int64

In [24]:
class Face_Race_Dataset(torch.utils.data.Dataset):
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.transform = torchvision.transforms.Compose(
                            [
                            np.float32,
                            torchvision.transforms.ToTensor(),
                            fixed_image_standardization])

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)

In [25]:
totalset = Face_Race_Dataset(data['face'].tolist(),data['race'].tolist())
dataset = Face_Race_Dataset(x,y)

In [26]:
train_size = int(0.9 * len(dataset))
valid_size = (len(dataset) - train_size)
train_set,valid_set = torch.utils.data.random_split(dataset,[train_size,valid_size])

# train_size = int(0.8 * len(dataset))
# valid_size = (len(dataset) - train_size)
# train_set,valid_size = torch.utils.data.random_split(dataset,[train_size,valid_size])

# valid_size = int(valid_size / 2)
# test_size = valid_size
# valid_set,test_set = torch.utils.data.random_split(valid_size,[valid_size,test_size])

In [27]:
train_loader = torch.utils.data.DataLoader(train_set,batch_size=32,num_workers=workers,shuffle=True)
#test_loader = torch.utils.data.DataLoader(test_set,batch_size=32,num_workers=workers,shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set,batch_size=32,num_workers=workers,shuffle=True)

# Define Inception Resnet V1 module

In [28]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=4
).to(device)

# Define optimizer, scheduler, loss, evaluation functions

In [29]:
optimizer = torch.optim.AdamW(resnet.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [5, 10])

loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

# Train model

In [30]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, valid_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, valid_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------
Valid |     2/2    | loss:    1.4625 | fps:  339.2374 | acc:    0.2203   

Epoch 1/16
----------


C:\Users\zhouy\AppData\Local\Temp/ipykernel_4008/2642461197.py:15: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)


Train |    18/18   | loss:    1.5749 | fps:  121.3645 | acc:    0.3729   
Valid |     2/2    | loss:  642.6880 | fps:  586.1047 | acc:    0.2218   

Epoch 2/16
----------
Train |    18/18   | loss:    1.3522 | fps:  135.1926 | acc:    0.4280   
Valid |     2/2    | loss:    9.3128 | fps:  583.1981 | acc:    0.4118   

Epoch 3/16
----------
Train |    18/18   | loss:    1.1830 | fps:  139.9443 | acc:    0.5054   
Valid |     2/2    | loss:    1.8553 | fps:  577.8148 | acc:    0.4123   

Epoch 4/16
----------
Train |    18/18   | loss:    1.0249 | fps:  143.0798 | acc:    0.5657   
Valid |     2/2    | loss:    1.3955 | fps:  601.6912 | acc:    0.4451   

Epoch 5/16
----------
Train |    18/18   | loss:    1.0261 | fps:  131.7797 | acc:    0.5854   
Valid |     2/2    | loss:    1.7553 | fps:  629.8549 | acc:    0.3805   

Epoch 6/16
----------
Train |    18/18   | loss:    0.9419 | fps:  139.5529 | acc:    0.6162   
Valid |     2/2    | loss:    1.2386 | fps:  636.3835 | acc:    0.4123 

In [31]:
torch.save(resnet.state_dict(),'./race_prediction_image/dataset1_pic_race_model.pt')

In [32]:
resnet.load_state_dict(torch.load('./race_prediction_image/dataset1_pic_race_model.pt'))
dataloader = torch.utils.data.DataLoader(totalset,batch_size=32,num_workers=workers,shuffle=False)
resnet.eval()
total_pred = []
total_y = []
for i, (x,y) in enumerate(dataloader):
    x = x.to(device)
    preds = resnet(x)
    _, pred_y = torch.max(preds,1)
    pred_y = pred_y.detach().cpu().numpy().tolist()
    y = y.detach().cpu().numpy().tolist()
    total_pred += pred_y
    total_y += y


C:\Users\zhouy\AppData\Local\Temp/ipykernel_4008/2642461197.py:15: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return self.transform(self.x[index]),torch.tensor(self.y[index],dtype=torch.long)


In [33]:
from sklearn import metrics
from sklearn.metrics import classification_report
print(classification_report(total_y, total_pred))

              precision    recall  f1-score   support

           0       0.97      0.56      0.71      1951
           1       0.40      0.94      0.56       224
           2       0.18      0.82      0.29       136
           3       0.31      0.42      0.36        74

    accuracy                           0.61      2385
   macro avg       0.46      0.68      0.48      2385
weighted avg       0.85      0.61      0.66      2385

